#### Importing Libraries

In [1]:
from sklearn.datasets import load_diabetes

import pandas as pd
import numpy as np
import time
import random

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#### Importing the Dataset

In [2]:
x,y = load_diabetes(return_X_y=True)

#### Splitting the data into train and test data

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

#### Training and finding the values used in the Model using Linear Regression

In [4]:
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

#### Accuracy of the model

In [5]:
y_pred = regressor.predict(x_test)
r2_score(y_test, y_pred)

0.4399387660024645

In [6]:
print(regressor.coef_)
print(regressor.intercept_)

[  -9.16088483 -205.46225988  516.68462383  340.62734108 -895.54360867
  561.21453306  153.88478595  126.73431596  861.12139955   52.41982836]
151.88334520854633


#### Making our own Mini Batch Gradient Descent class

In [7]:
class MyMiniBatchGradientDescent:

    def __init__(self, batch_size, learning_rate = 0.01, epochs = 100):
        # Initialize the coefficients and intercept to None
        self.coef_ = None
        self.intercept_ = None
        # Set the batch_size, learning rate and number of epochs for training
        self.batch_size = batch_size
        self.lr = learning_rate
        self.epochs = epochs 
    
    def fit(self, x_train, y_train):
        # Initialize the intercept to 0 and the coefficients to ones
        self.intercept_ = 0
        self.coef_ = np.ones(x_train.shape[1])
        
        # Loop through the number of epochs specified
        for i in range(self.epochs):
            for j in range(int(x_train.shape[0]/self.batch_size)):
                rand = random.sample(range(x_train.shape[0]), self.batch_size)
                y_cap = np.dot(x_train[rand], self.coef_) + self.intercept_

                # Calculate the derivative of the intercept with respect to the loss function
                intercept_derivative = -2 * np.mean(y_train[rand] - y_cap)
                # Update the intercept using the learning rate and derivative
                self.intercept_ = self.intercept_ - (self.lr * intercept_derivative)

                # Calculate the derivative of the coefficients with respect to the loss function
                coef_derivative = -2 * np.dot((y_train[rand] - y_cap), x_train[rand])
                # Update the coefficients using the learning rate and derivative
                self.coef_ = self.coef_ - (self.lr * coef_derivative)
            
        # Print the final intercept and coefficients
        print(self.intercept_,self.coef_)
    
    def predict(self, x_test):
        # Use the final coefficients and intercept to make predictions on new data
        return np.dot(x_test, self.coef_) + self.intercept_

#### Accuracy of Mini Batch Gradient Descent class

In [8]:
regressor2 = MyMiniBatchGradientDescent(batch_size=int(x_train.shape[0]/10), epochs=50,learning_rate=0.01)

In [9]:
start = time.time()
regressor2.fit(x_train, y_train)
print()
print("The time taken is", time.time() - start)

151.77791577030277 [  59.57754452  -67.45657313  344.29513993  258.06982021   12.36253324
  -39.03519179 -166.40232689  119.02730113  322.0205225   122.72319423]

The time taken is 0.06454014778137207


In [10]:
y_pred2 = regressor2.predict(x_test)
r2_score(y_test, y_pred2)

0.431943584968854

#### Using SGD Regressor to apply Mini Batch Gradient Descent

In [11]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)

In [12]:
batch_size = 35   # Initialize the batch size to 35

for i in range(100):   # Loop through 100 iterations
    rand = random.sample(range(x_train.shape[0]), batch_size)   # Randomly select 'batch_size' number of samples from the training set
    sgd.partial_fit(x_train[rand], y_train[rand])   # Fit the stochastic gradient descent model on the selected samples

In [13]:
y_pred3 = sgd.predict(x_test)

In [14]:
r2_score(y_test, y_pred3)

0.4185915963842659